In [1]:
from splinter import Browser
from splinter.driver.webdriver import WebDriverElement
from selenium import webdriver
import time, re

In [2]:
def select_option_by_value(select:WebDriverElement, val:str)->type(None):
    pattern = re.compile(r"^.*(){str}$".format(str=val))
    select.click()
    options = select.find_by_tag('option')
    for i in range(0, len(options)):
        option = options[i]
        if pattern.match(option.value):
            option.click()
            break

In [3]:
# Init Settings
url = "https://ieeexplore.ieee.org/Xplore/home.jsp"
is_headless = True
search_opts = {
    'page-delay': 3,
    'item-1': {
        'query': "web scraping",
        'option': "Abstract"
    },
    'item-2': {
        'query': "python",
        'option': "Abstract"
    },
    'from': 2010,
    'to': 2021
}

# Chrome Driver Settings
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
cdriver_path = { 'executable_path':r'C:\Users\devwarlt\Desktop\Faculdade\UNB - Pesquisador (arqs)\Web Scraping\chromedriver' }

# Browser Settings
browser = Browser("chrome", **cdriver_path, headless = is_headless, options = options)

In [4]:
# Access page
browser.visit(url)

In [5]:
# Remove cookie banner
browser.find_by_css('div[class="cc-compliance"] a[class="cc-btn cc-dismiss"]').click()

In [6]:
# Click on Advanced Search
browser.links.find_by_partial_href('advanced').click()

In [7]:
# Awaits until IEEE loads rest of forms
time.sleep(search_opts['page-delay'])

In [8]:
forms = browser.find_by_tag('form')
print("total forms -> {}".format(len(forms)))

total forms -> 3


In [9]:
# Get valid inputs & selects
adv_form = forms.last
adv_form_inputs = adv_form.find_by_tag('input')
adv_form_selects = adv_form.find_by_tag('select')
print("total inputs -> {}".format(len(adv_form_inputs)))
print("total selects -> {}".format(len(adv_form_selects)))

total inputs -> 7
total selects -> 5


In [10]:
# Fill specific inputs
adv_form_inputs[0].fill(search_opts['item-1']['query']);
adv_form_inputs[1].fill(search_opts['item-2']['query']);
print("input (type: {})".format(type(adv_form_inputs[0])))
print("select (type: {})".format(type(adv_form_selects[0])))

input (type: <class 'splinter.driver.webdriver.WebDriverElement'>)
select (type: <class 'splinter.driver.webdriver.WebDriverElement'>)


In [11]:
# Select specific options
select_option_by_value(adv_form_selects[0], search_opts['item-1']['option']);
select_option_by_value(adv_form_selects[2], search_opts['item-2']['option']);

In [12]:
# Get specific elements to remove
adv_form_to_rmv = adv_form.find_by_css('div i[class="fa fa-times"]')
print("total X btns -> {}".format(len(adv_form_to_rmv)))

total X btns -> 2


In [13]:
# Remove specific element
adv_form_to_rmv.last.click()

In [14]:
# Chose 'Specify Year Range' option
adv_form.find_by_xpath('//span[text()="Specify Year Range"]').click()

In [15]:
# Get from-to input fields
adv_syr_inputs = adv_form.find_by_css('span[class="row"] span input[type="text"]')
print("total from-to inputs -> {}".format(len(adv_syr_inputs)))

total from-to inputs -> 2


In [16]:
adv_syr_inputs[0].fill(search_opts['from'])
adv_syr_inputs[1].fill(search_opts['to'])

In [17]:
# Search
adv_form.find_by_css('div[class="submit-box"] button[type="submit"]').click()

In [18]:
# Awaits until IEEE render search query
time.sleep(search_opts['page-delay'])

In [19]:
# Count results
query_results = browser.find_by_css('div[class="List-results-items"]')
total_results = 0
query_spans = browser.find_by_css(
    'section div[class="Dashboard-section Dashboard-section-gray"] ' +
    'div[class="Dashboard-header col-12"] span span[class="strong"]'
)
if len(query_spans) > 1:
    total_results = int(query_spans[1].text.replace(',', ''))
print("{} Displaying {} of {}".format(
    "Found results!" if total_results > 0 else "No results found...",
    len(query_results),
    total_results
    )
)

Found results! Displaying 12 of 12


In [20]:
# Close browser
browser.quit()